## A. Configuraciones Generales.

In [ ]:
#1. Librerías.
%run "../librerias.ipynb"

In [ ]:
#2. Constantes.
#a. Constantes generales.
%run "../constantes.ipynb"

#b. Constantes a definir por el usuario.
#Información sobre rango temporal del modelo.
cantidad_meses_train = "all"
ventana = 1
#Meses de train y test.
mes_train = mes_train_all_menos_2_sin_rotas_backtesting
mes_test = mes_test_backtesting

#i. Modelo 1.
#1. Dataset de lectura (post-feature engineering y undersampleado).
dataset_con_fe_m1 = dataset_file_fe_1_all_undersampleado
dataset_con_test_m1 = dataset_file_fe_all_1
#2. Ruta de la BBDD donde se almacenan los hiperparámetros óptimos post-Optuna.
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name_m1 = f"exp_lgbm_comp3_{cantidad_meses_train}_{ventana}_undersampling_backtesting"

#ii. Modelo 2.
#1. Dataset de lectura (post-feature engineering y undersampleado).
dataset_con_fe_m2 = dataset_file_fe_1_all_undersampleado_lagsdelta
dataset_con_test_m2 = dataset_file_fe_all_1_lagsdelta
#2. Ruta de la BBDD donde se almacenan los hiperparámetros óptimos post-Optuna.
study_name_m2 = f"exp_lgbm_comp3_{cantidad_meses_train}_{ventana}_undersampling_backtesting_lagsdelta"

In [ ]:
#3. Lectura de datos.
#i. Modelo 1.
#a. Train.
data_m1 = pd.read_parquet(dataset_con_fe_m1)
#b. Test.
test_m1 = pd.read_parquet(dataset_con_test_m1)

#ii. Modelo 2.
#a. Train.
data_m2 = pd.read_parquet(dataset_con_fe_m2)
#b. Test.
test_m2 = pd.read_parquet(dataset_con_test_m2)

In [ ]:
#4. Importo el estudio de Optuna.
#i. Modelo 1.
study_m1 = optuna.load_study(
    study_name=study_name_m1,
    storage=storage_name
    )

#ii. Modelo 2.
study_m2 = optuna.load_study(
    study_name=study_name_m2,
    storage=storage_name
    )

## B. Pre-procesamiento.

In [ ]:
#1. Concateno Train y Test.
#i. Modelo 1.
#a. Filtro el mes de interés en Test.
test_m1 = test_m1[test_m1["foto_mes"] == mes_test]
#b. Borro la columna de clase binaria en train para poder concatenar.
data_m1.drop(["clase_binaria"],axis=1,inplace=True)
#c. Concateno.
data_m1 = pd.concat([data_m1,test_m1],axis=0)

#ii. Modelo 2.
#a. Filtro el mes de interés en Test.
test_m2 = test_m2[test_m2["foto_mes"] == mes_test]
#b. Borro la columna de clase binaria en train para poder concatenar.
data_m2.drop(["clase_binaria"],axis=1,inplace=True)
#c. Concateno.
data_m2 = pd.concat([data_m2,test_m2],axis=0)


In [ ]:
#2. Pequeño pre-procesamiento sobre los datos.
#i. Modelo 1.
#a. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data_m1['ctrx_quarter_normalizado'] = data_m1['ctrx_quarter_normalizado'].astype(float)

#b. Pesos y reclusterización.
data_m1['clase_peso'] = 1.0

data_m1.loc[data_m1['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data_m1.loc[data_m1['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data_m1['clase_binaria'] = 0
data_m1['clase_binaria'] = np.where(data_m1['clase_ternaria'] == 'CONTINUA', 0, 1)

#ii. Modelo 2.
#a. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data_m2['ctrx_quarter_normalizado'] = data_m2['ctrx_quarter_normalizado'].astype(float)

#b. Pesos y reclusterización.
data_m2['clase_peso'] = 1.0

data_m2.loc[data_m2['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data_m2.loc[data_m2['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data_m2['clase_binaria'] = 0
data_m2['clase_binaria'] = np.where(data_m2['clase_ternaria'] == 'CONTINUA', 0, 1)

In [ ]:
#3. Dividimos entre conjuntos de datos.
#i. Modelo 1.
#a. Datos para entrenar todo el modelo final para Kaggle.
train_data_m1 = data_m1[data_m1['foto_mes'].isin(mes_train)]
test_data_m1 = data_m1[data_m1['foto_mes'] == mes_test]

X_train_m1 = train_data_m1.drop(['clase_ternaria', 'clase_peso','clase_binaria'], axis=1)
y_train_binaria_m1 = train_data_m1['clase_binaria']
w_train_m1 = train_data_m1['clase_peso']

#b. Datos de Test (a predecir).
X_test_m1 = test_data_m1.drop(['clase_ternaria', 'clase_peso','clase_binaria'], axis=1)
y_test_binaria_m1 = test_data_m1['clase_binaria']

#ii. Modelo 2.
#a. Datos para entrenar todo el modelo final para Kaggle.
train_data_m2 = data_m2[data_m2['foto_mes'].isin(mes_train)]
test_data_m2 = data_m2[data_m2['foto_mes'] == mes_test]

X_train_m2 = train_data_m2.drop(['clase_ternaria', 'clase_peso','clase_binaria'], axis=1)
y_train_binaria_m2 = train_data_m2['clase_binaria']
w_train_m2 = train_data_m2['clase_peso']

#b. Datos de Test (a predecir).
X_test_m2 = test_data_m2.drop(['clase_ternaria', 'clase_peso','clase_binaria'], axis=1)
y_test_binaria_m2 = test_data_m2['clase_binaria']


## C. Entrenamiento simple.

In [ ]:
#a. Modelo 1.
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study_m1.best_trial.params['num_leaves'],
    'learning_rate': study_m1.best_trial.params['learning_rate'],
    'min_data_in_leaf': study_m1.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study_m1.best_trial.params['feature_fraction'],
    'bagging_fraction': study_m1.best_trial.params['bagging_fraction'],
    'seed': semillas[0],
    'verbose': 0
}

train_data_m1 = lgb.Dataset(X_train_m1,
                          label=y_train_binaria_m1,
                          weight=w_train_m1)

modelo_1 = lgb.train(params,
                  train_data_m1,
                  num_boost_round=best_iter)

In [ ]:
#b. Modelo 2.
best_iter = study.best_trial.user_attrs["best_iter"]
print(f"Mejor cantidad de árboles para el mejor model {best_iter}")
params = {
    'objective': 'binary',
    'boosting_type': 'gbdt',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_bin': 31,
    'num_leaves': study_m2.best_trial.params['num_leaves'],
    'learning_rate': study_m2.best_trial.params['learning_rate'],
    'min_data_in_leaf': study_m2.best_trial.params['min_data_in_leaf'],
    'feature_fraction': study_m2.best_trial.params['feature_fraction'],
    'bagging_fraction': study_m2.best_trial.params['bagging_fraction'],
    'seed': semillas[0],
    'verbose': 0
}

train_data_m2 = lgb.Dataset(X_train_m2,
                          label=y_train_binaria_m2,
                          weight=w_train_m2)

modelo_2 = lgb.train(params,
                  train_data_m2,
                  num_boost_round=best_iter)

## D. Predicciones.

In [ ]:
#1. Predigo Junio.
print("Empieza el Modelo 1...")
y_pred_m1 = modelo_1.predict(X_test_m1)
print("Empieza el Modelo 2...")
y_pred_m2 = modelo_2.predict(X_test_m2)

## E. Análisis.

In [ ]:
#1. Medimos las ganancias de los modelos.
def ganancia_prob(y_pred, y_true, prop = 1):
  ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
  return ganancia[y_pred >= 0.025].sum() / prop

print("Ganancia M1:", ganancia_prob(y_pred_m1, y_test_binaria_m1))
print("Ganancia M2:", ganancia_prob(y_pred_m2, y_test_binaria_m2))

In [ ]:
#2. ¿Y si hacemos distintas divisiones entre Público y Privado?
#i. Estratificamos 50 escenarios distintos.
sss_futuro = StratifiedShuffleSplit(n_splits=50,
                                    test_size=0.1,
                                    random_state=semillas[0])

#ii. Definimos los modelos y sus respectivos X_test y y_test.
modelos = {
    "m1": {"pred": y_pred_m1, "X_test": X_test_m1, "y_test": y_test_binaria_m1},
    "m2": {"pred": y_pred_m2, "X_test": X_test_m2, "y_test": y_test_binaria_m2},
}

rows = []

#iii. Para cada modelo, aplicamos StratifiedShuffleSplit.
for name, data in modelos.items():
    X_test = data["X_test"]
    y_test = data["y_test"]
    y_pred = data["pred"]
    
    for private_index, public_index in sss_futuro.split(X_test, y_test):
        row = {}
        # Calculamos la ganancia en la división pública y privada.
        row[name + "_public"] = ganancia_prob(y_pred[public_index], y_test.iloc[public_index], 0.1)
        row[name + "_private"] = ganancia_prob(y_pred[private_index], y_test.iloc[private_index], 0.9)
        rows.append(row)

#iv. Convertimos a DataFrame.
df_lb = pd.DataFrame(rows)

#v. Transformación a formato largo para visualización.
df_lb_long = df_lb.reset_index()
df_lb_long = df_lb_long.melt(id_vars=['index'], var_name='model_type', value_name='ganancia')
df_lb_long[['modelo', 'tipo']] = df_lb_long['model_type'].str.split('_', expand=True)
df_lb_long = df_lb_long[['ganancia', 'tipo', 'modelo']]

#vi. Visualización con seaborn.
g = sns.FacetGrid(df_lb_long, col="tipo", row="modelo", aspect=2)
g.map(sns.histplot, "ganancia", kde=True)
plt.show()

In [ ]:
#3. Entender como se hubiese comportado el Público y Privado a partir de distintos puntos de corte de cantidad de envíos.
#i. Función.
def analisis_comportamiento_kaggle_completo(semilla, desde, paso, cantidad, private = False,ganancia = None,y_pred = None, y_etiqueta = None):

  df_cut_point = pd.DataFrame({'ganancia': ganancia, 'y_pred_lgm': y_pred})
  df_cut_point['nro_envios'] = df_cut_point.reset_index().index

  plt.figure(figsize=(10, 6))

  private_idx, public_idx = train_test_split(df_cut_point.index, test_size=0.1, random_state=semilla, stratify=y_etiqueta)

  df_cut_point['public'] = 0.0
  df_cut_point.loc[public_idx, 'public'] = ganancia[public_idx] / 0.1
  df_cut_point['public_cum'] = df_cut_point['public'].cumsum()

  maximo_paso = desde + paso*cantidad
  plt.plot(df_cut_point['nro_envios'][list(range(desde, maximo_paso + 1, paso))], df_cut_point['public_cum'][list(range(desde, maximo_paso + 1, paso))], label='Ganancia Pública Acumulada')
  max_public_cum = df_cut_point['public_cum'][list(range(desde, maximo_paso + 1, paso))].max()
  max_public_idx = df_cut_point['public_cum'][list(range(desde, maximo_paso + 1, paso))].idxmax()
  plt.axvline(x=max_public_idx, color='g', linestyle='--', label=f'Máximo Ganancia Pública en {max_public_idx}')

  if private:
    df_cut_point['private'] = 0.0
    df_cut_point.loc[private_idx, 'private'] = ganancia[private_idx] / 0.9
    df_cut_point['private_cum'] = df_cut_point['private'].cumsum()
    plt.plot(df_cut_point['nro_envios'][4000:20000], df_cut_point['private_cum'][4000:20000], label='Ganancia Privada Acumulada')
    max_private_cum = df_cut_point['private_cum'][4000:20000].max()
    max_private_idx = df_cut_point['private_cum'][4000:20000].idxmax()
    plt.axvline(x=max_private_idx, color='r', linestyle='--', label=f'Máximo Ganancia Privada en {max_private_idx}')

  plt.title('Curva de Ganancia Pública y Privada')
  plt.xlabel('Número de envíos')
  plt.ylabel('Ganancia Acumulada')
  plt.legend()
  plt.show()

In [ ]:
#ii. Calculo de ganancia acumulada.
#a. Modelo 1.
ganancia_m1 = np.where(y_test_binaria_m1 == 1, ganancia_acierto, 0) - np.where(y_test_binaria_m1 == 0, costo_estimulo, 0)

idx_m1 = np.argsort(y_pred_m1)[::-1]

ganancia_m1 = ganancia_m1[idx_m1]
y_pred_m1 = y_pred_m1[idx_m1]

ganancia_cum_m1 = np.cumsum(ganancia_m1)

#b. Modelo 2.
ganancia_m2 = np.where(y_test_binaria_m2 == 1, ganancia_acierto, 0) - np.where(y_test_binaria_m2 == 0, costo_estimulo, 0)

idx_m2 = np.argsort(y_pred_m2)[::-1]

ganancia_m2 = ganancia_m2[idx_m2]
y_pred_m2 = y_pred_m2[idx_m2]

ganancia_cum_m2 = np.cumsum(ganancia_m2)

In [ ]:
#ii. Ejecución para el modelo 1.
analisis_comportamiento_kaggle_completo(semillas[0],9000,100,50,private=True,ganancia=ganancia_m1,y_pred=y_pred_m1,y_etiqueta=y_test_binaria_m1)

In [ ]:
#iii. Ejecución para el modelo 2.
analisis_comportamiento_kaggle_completo(semillas[0],4000,500,32,private=True,ganancia=ganancia_m2,y_pred=y_pred_m2,y_etiqueta=y_test_binaria_m2)